In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, PowerTransformer, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# ==========================================
# 1. LOAD DATA
# ==========================================
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

# ==========================================
# 2. FEATURE ENGINEERING
# ==========================================
def create_features(df):
    df = df.copy()
    # 1. Activity Score
    df['activity_score'] = (df['hobby_engagement_level'] + 
                            df['physical_activity_index'] + 
                            df['creative_expression_index'])
    # 2. Support Score
    df['support_total'] = (df['support_environment_score'] + 
                           df['external_guidance_usage'] + 
                           df['upbringing_influence'])
    # 3. Efficiency Ratios
    df['efficiency'] = df['consistency_score'] / (df['focus_intensity'] + 1.0)
    df['focus_per_support'] = df['focus_intensity'] / (df['support_environment_score'] + 1.0)
    # 4. Age Norms
    df['consistency_per_age'] = df['consistency_score'] / df['age_group']
    return df

print("Feature Engineering...")
train_eng = create_features(train)
test_eng = create_features(test)

# ==========================================
# 3. PCA-GUIDED CLUSTERING
# ==========================================
print("Generating PCA-Cluster Features...")

full_data = pd.concat([train_eng.drop('personality_cluster', axis=1), test_eng], axis=0, ignore_index=True)

# Features defining structure
cluster_cols = ['focus_intensity', 'consistency_score', 'efficiency', 'activity_score', 'support_total', 'focus_per_support']
scaler = StandardScaler()
full_scaled = scaler.fit_transform(full_data[cluster_cols])

# PCA to remove noise
pca = PCA(n_components=0.95, random_state=42)
full_pca = pca.fit_transform(full_scaled)

# Cluster features (The "GPS coordinates" for KNN)
kmeans_5 = KMeans(n_clusters=5, random_state=42, n_init=50)
full_data['cluster_pca_5'] = kmeans_5.fit_predict(full_pca)

kmeans_8 = KMeans(n_clusters=8, random_state=42, n_init=50)
full_data['cluster_pca_8'] = kmeans_8.fit_predict(full_pca)

# Split back
train_eng['cluster_pca_5'] = full_data.iloc[:len(train)]['cluster_pca_5'].values
train_eng['cluster_pca_8'] = full_data.iloc[:len(train)]['cluster_pca_8'].values
test_eng['cluster_pca_5'] = full_data.iloc[len(train):]['cluster_pca_5'].values
test_eng['cluster_pca_8'] = full_data.iloc[len(train):]['cluster_pca_8'].values

# ==========================================
# 4. PREPROCESSING PIPELINE
# ==========================================
target_col = 'personality_cluster'
X = train_eng.drop([target_col, 'participant_id'], axis=1)
y = train_eng[target_col]
X_test = test_eng.drop(['participant_id'], axis=1)

nominal_cols = ['cultural_background', 'cluster_pca_5', 'cluster_pca_8']
numeric_cols = [c for c in X.columns if c not in nominal_cols]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('power', PowerTransformer(method='yeo-johnson')), # fixes skew
    ('selector', SelectPercentile(f_classif, percentile=90)) # remove noise
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, nominal_cols)
    ]
)

# ==========================================
# 5. OPTIMIZED PURE KNN
# ==========================================
# We use a Pipeline to prevent data leakage during GridSearch
knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', KNeighborsClassifier())
])

# Tuning Grid:
# 'weights': ['distance'] is usually better for high K
# 'p': 1 (Manhattan) often works better on high-dimensional data than 2 (Euclidean)
param_grid = {
    'knn__n_neighbors': [15, 20, 25, 30, 35],
    'knn__weights': ['distance'], 
    'knn__p': [1, 2] 
}

print("Tuning KNN...")
grid = GridSearchCV(knn_pipeline, param_grid, cv=5, scoring='f1_macro', n_jobs=-1)
grid.fit(X, y)

print(f"Best KNN Score: {grid.best_score_}")
print(f"Best KNN Params: {grid.best_params_}")

# ==========================================
# 6. SUBMISSION
# ==========================================
preds = grid.predict(X_test)

submission = pd.DataFrame({
    'participant_id': test['participant_id'],
    'personality_cluster': preds
})

submission.to_csv('submission_pure_knn_optimized.csv', index=False)
print("Saved: submission_pure_knn_optimized.csv")
print(submission.head())

Feature Engineering...
Generating PCA-Cluster Features...
Tuning KNN...
Best KNN Score: 0.48691593305401987
Best KNN Params: {'knn__n_neighbors': 15, 'knn__p': 1, 'knn__weights': 'distance'}
Saved: submission_pure_knn_optimized.csv
   participant_id personality_cluster
0            1005           Cluster_E
1             197           Cluster_C
2            2343           Cluster_E
3            1709           Cluster_A
4             436           Cluster_E
